# Package Imports

In [1]:
import pandas as pd
from pathlib import Path
import time
import tqdm
from datetime import datetime
import os
from sklearn.model_selection import KFold
import sys
import torch.nn.functional as F
import pytorch_lightning as plit
import pandas as pd
import random
import math
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, normalize, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_recall_curve, confusion_matrix, roc_curve
from sklearn.metrics import auc as skauc
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from fancyimpute import IterativeImputer as MICE
from sklearn.compose import ColumnTransformer
import numpy as np
from multiprocessing import Pool
import multiprocessing
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from xgboost import XGBClassifier, XGBRegressor
import torch.nn as nn
import shap
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, TensorDataset
import torch
import argparse
import dateutil
from collections import defaultdict
from scipy.stats import gaussian_kde
from sklearn.neighbors import NearestNeighbors

# Torch device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# K-fold validation
n_splits = 5  
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

/home/defrondeville.c/miniconda3/envs/LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Autoencoder for Patient Codes

In [2]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        
        # Encoder layers
        self.encoder_fc1 = nn.Linear(input_dim, 256)
        self.encoder_fc2_mean = nn.Linear(256, latent_dim)  # Output mean
        self.encoder_fc2_logvar = nn.Linear(256, latent_dim)  # Output log variance
        
        # Decoder layers
        self.decoder_fc1 = nn.Linear(latent_dim, 256)
        self.decoder_fc2 = nn.Linear(256, input_dim)

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def encode(self, x):
        x = F.relu(self.encoder_fc1(x))
        return self.encoder_fc2_mean(x), self.encoder_fc2_logvar(x)

    def decode(self, z):
        x = F.relu(self.decoder_fc1(z))
        return torch.sigmoid(self.decoder_fc2(x))

    def forward(self, x):
        # Encoding
        z_mean, z_log_var = self.encode(x)

        # Sampling
        z = self.reparameterize(z_mean, z_log_var)

        # Decoding
        reconstructed = self.decode(z)
        
        return reconstructed, z_mean, z_log_var
    
    def infer(self, x):
        # Encoding
        z_mean, _ = self.encode(x)
        z_mean = z_mean.detach().numpy()[0]
        return z_mean
        
class AutoEncoder:
    def __init__(self, ICD = 'dataset/DIAGNOSES_ICD.csv',
                 conversion = 'dataset/D_ICD_DIAGNOSES.csv',
                 max_length = 100,
                 latent_dim = 1):
        
        # Loads data
        self.conversion = pd.read_csv(conversion)
        self.ICD = pd.read_csv(ICD)

        tokens = set(self.conversion['ICD9_CODE'])
        self.code_converter = {item:i for i, item in zip(range(len(tokens)), tokens)}
        subject_codes = defaultdict(list)

        for num, row in tqdm.tqdm(self.ICD.iterrows(), total = len(self.ICD), desc = 'Creating code features'):
            subject_id = row['SUBJECT_ID']
            diagnoses = row['ICD9_CODE']
            try:
                subject_codes[subject_id].append(code_converter[diagnoses])
            except:
                self.code_converter[diagnoses] = len(self.code_converter)
                subject_codes[subject_id].append(self.code_converter[diagnoses])
                
        subject_codes = dict(subject_codes)
        self.subjects = list(subject_codes.keys())

        # Obtains tensors
        self.input_tensors = []
        self.max_length = max_length
        for num, (subject_id, codes) in tqdm.tqdm(enumerate(subject_codes.items()), total = len(subject_codes), desc = 'Padding samples'):
            while len(codes) < max_length:
                codes.append(0)
            if len(codes) > max_length:
                codes = random.sample(codes, max_length)
            self.input_tensors.append(torch.FloatTensor(codes))

        # Scales data
        self.scaler = StandardScaler()
        self.input_tensors = torch.FloatTensor(self.scaler.fit_transform(self.input_tensors))
        
        # Makes VAE
        self.autoencoder = VAE(input_dim = self.max_length, latent_dim = latent_dim)


    def train_VAE(self, batch_size = 32,
                 num_epochs = 10):
        
        # Define the DataLoader
        batch_size = 16
        data_loader = DataLoader(self.input_tensors, batch_size=batch_size, shuffle=True)

        # Define the loss function and optimizer
        def loss_function(reconstructed_x, x, mu, log_var):
            # Reconstruction loss (MSE)
            recon_loss = F.mse_loss(reconstructed_x, x) / x.size(0)  # Divide by batch size for stability
            
            # KL divergence
            kld_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp()) / x.size(0)
        
            return recon_loss + kld_loss

        optimizer = torch.optim.Adam(self.autoencoder.parameters(), lr=0.0001)

        # Training loop
        self.autoencoder.train()
        for epoch in range(num_epochs):
            total_loss = 0.0
            for i, data in enumerate(tqdm.tqdm(data_loader, total=len(data_loader), desc=f'Epoch {epoch}')):
                optimizer.zero_grad()
                outputs, mu, log_var = self.autoencoder(data)
                loss = loss_function(outputs, data, mu, log_var)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(data_loader):.4f}')

        # Obtains encoded features
        encoded_features = []
        for feature in tqdm.tqdm(self.input_tensors, total = len(self.input_tensors), desc = 'Encoding...'):
            feature = self.autoencoder.infer(feature)
            encoded_features.append(feature)
        return encoded_features

    def extract_features(self, data, scale = True):
        data = np.array(data)
        self.autoencoder.eval()
        
        if len(data) < self.max_length:    
            while len(data) < self.max_length:
                data = np.append(data, 0)
                
        elif len(data) > self.max_length:
            data = data[self.max_length]

        data = self.scaler.transform([data])
        
        outputs = self.autoencoder.infer(torch.FloatTensor(data)).detach().numpy()[0]
        return outputs


# Feature Analysis + Filtration

In [3]:
def analyze_features(features_scaled, # Feature dataframe
                     feature_names, # Binary - on drug or not on drug EVERY DRUG NAME [i for i in features.columns if 'dose' in i] 
                     y, # Mortality labels 
                     top_n_features=10, # What features will be plotted (shap, LR, PCA) 
                     return_top_features = False ): # Return a list of filtered feature name  
    """
    Analyze features using SHAP values, Linear Regression, and Random Forest, 
    select the top features from each method without duplicates, and visualize the results.

    :param features_scaled: Scaled feature numpy array
    :param feature_names: List of feature names
    :param y: True labels
    :param top_n_features: Number of top features to retain from each method
    """
    # Trains XGBoost classifier for SHAP
    xgb_model = XGBClassifier()
    xgb_model.fit(features_scaled, y)
    
    # Trains Random Forest classifier
    rf_model = RandomForestClassifier()
    rf_model.fit(features_scaled, y)

    # Linear Regression for feature importance
    lr_model = LinearRegression()
    lr_model.fit(features_scaled, y)

    # SHAP values
    explainer_shap = shap.Explainer(xgb_model)
    shap_values = explainer_shap(features_scaled)
    sampled_features = shap.sample(features_scaled, 1000)
    explainer_tree = shap.TreeExplainer(xgb_model)
    shap_values_tree = explainer_tree.shap_values(sampled_features)

    # Feature importance from SHAP
    feature_importance_shap = np.abs(shap_values.values).mean(axis=0)
    top_features_shap = np.argsort(feature_importance_shap)[-top_n_features:]

    # Feature importance from RF
    feature_importance_rf = rf_model.feature_importances_
    top_features_rf = np.argsort(feature_importance_rf)[-top_n_features:]

    # Feature importance from LR
    feature_importance_lr = np.abs(lr_model.coef_)
    top_features_lr = np.argsort(feature_importance_lr)[-top_n_features:]

    # Visualization
    
    # Visualize SHAP values using summary plot
    shap.summary_plot(shap_values_tree, sampled_features, feature_names=feature_names)

    # Print feature importance based on SHAP
    feature_importance_shap_tree = np.abs(shap_values_tree).mean(axis=0)
    feature_importance_shap_tree = pd.Series(feature_importance_shap_tree, index=feature_names).sort_values(ascending=False)
    print("\nFeature importance based on SHAP TreeExplainer:")
    print(feature_importance_shap_tree.head(top_n_features))
    print("\n" + "-"*50 + "\n")

    # RF
    plt.figure(figsize=(10, 6))
    indices = np.argsort(feature_importance_rf)[::-1][:top_n_features]
    plt.title("Top Random Forest Feature Importances")
    plt.barh(range(top_n_features), feature_importance_rf[indices], color='b', align='center')
    plt.yticks(range(top_n_features), [feature_names[i] for i in indices])
    plt.xscale('log')
    plt.gca().invert_yaxis()  # Invert y-axis to have largest values at the top

    # LR
    plt.figure(figsize=(10, 6))
    indices = np.argsort(feature_importance_lr)[::-1][:top_n_features]
    plt.title("Top Linear Regression Coefficients")
    plt.barh(range(top_n_features), feature_importance_lr[indices], color='b', align='center')
    plt.yticks(range(top_n_features), [feature_names[i] for i in indices])
    plt.xscale('log')
    plt.gca().invert_yaxis()  # Invert y-axis to have largest values at the top

    # Combine and remove duplicates
    combined_feature_indices = np.unique(np.concatenate((top_features_shap, top_features_rf, top_features_lr)))
    combined_feature_names = [feature_names[i] for i in combined_feature_indices]

    print(f"Number of selected features: {len(combined_feature_names)}")
    print("Selected features:", combined_feature_names)

    if return_top_features:
        return combined_feature_names


def process_data_chunk_wrapper(chunk, id_conversion):
    """
    Calls process_data_chunk with the given chunk and id_conversion.
    """
    return process_data_chunk(chunk, id_conversion)
    
# Filters features based on their presence in the final patient dataset
def filter_features_on_presence(feature_df, feature_threshold =0.5, patient_threshold = 0.5):
    presence = feature_df.notna().mean()
    filtered_columns = presence[presence > feature_threshold].index
    feature_df = feature_df[filtered_columns]
    
    # Calculate the minimum number of non-NaN/None values required per row based on the proportion
    min_non_nan_count = int(np.ceil(patient_threshold * feature_df.shape[1]))
    
    # Filter rows based on the calculated minimum count of non-NaN/None values
    filtered_df = feature_df.dropna(axis=0, thresh=min_non_nan_count)
    
    return filtered_df
    
def filter_chart_data_optimized(chart_data, admissions_data, max_hours=48, chunks=4):
    # Ensure datetime format
    chart_data['CHARTTIME'] = pd.to_datetime(chart_data['CHARTTIME'], errors='coerce')
    admissions_data['ADMITTIME'] = pd.to_datetime(admissions_data['ADMITTIME'], errors='coerce')

    # Determine chunk size
    chunk_size = len(chart_data) // chunks
    filtered_data_list = []

    for i in range(chunks):
        # Calculate start and end index for each chunk
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if i < chunks - 1 else len(chart_data)
        chart_data_chunk = chart_data.iloc[start_idx:end_idx]

        # Merge with admissions_data
        merged_data = chart_data_chunk.merge(admissions_data[['SUBJECT_ID', 'ADMITTIME']],
                                             on='SUBJECT_ID', how='inner')
        # Calculate time difference in hours
        merged_data['time_diff'] = (merged_data['CHARTTIME'] - merged_data['ADMITTIME']).dt.total_seconds() / 3600

        # Filter based on time_diff
        filtered_chunk = merged_data.query('0 <= time_diff <= @max_hours').drop(columns=['ADMITTIME', 'time_diff'])
        filtered_data_list.append(filtered_chunk)

    # Concatenate all filtered chunks
    filtered_data = pd.concat(filtered_data_list, ignore_index=True)
    
    return filtered_data
    
def extract_features_from_chart_data(chart_data, id_conversion, num_processes=16, min_non_nan_proportion=0.01):
    """
    Extract features from chart data using multiprocessing.
    """
    # Split data into chunks
    chunks = np.array_split(chart_data, num_processes)
    
    # Prepare arguments for each chunk processing
    args = [(chunk, id_conversion) for chunk in chunks]
    
    # Process each chunk in parallel
    with Pool(num_processes) as pool:
        results = pool.starmap(process_data_chunk_wrapper, args)
    
    # Combine results from all chunks
    combined = pd.concat(results, axis=0).reset_index(drop=True).groupby('SUBJECT_ID').first()
    
    # Determine the threshold number of non-NaN/non-None values required per column
    threshold_count = int(min_non_nan_proportion * len(combined))
    
    # Drop columns where the number of non-NaN/non-None values is less than the threshold
    filtered_combined = combined.dropna(axis=1, thresh=threshold_count, inplace=False)
    
    return filtered_combined 
    
def process_data_chunk(chunk, id_conversion, value_pick='last'):
    """
    Extract features from a chunk of data, preserving SUBJECT_ID.
    Args:
        - chunk (DataFrame): A chunk of the filtered chart data, preserving SUBJECT_ID.
        - id_conversion (dict): A dictionary for converting item IDs to feature names.
        - value_pick (str): Determines whether to pick the 'first' or 'last' non-empty value for each feature.
    Returns:
        - DataFrame with extracted features for the chunk, including SUBJECT_ID.
    """

    # Ensure SUBJECT_ID is considered during processing
    chunk['FEATURE_NAME'] = chunk['ITEMID'].map(id_conversion).fillna('Unknown')

    # Define aggregation function based on value_pick parameter
    if value_pick == 'first':
        value_func = lambda x: x.dropna().iloc[0] if not x.dropna().empty else None
    elif value_pick == 'last':
        value_func = lambda x: x.dropna().iloc[-1] if not x.dropna().empty else None
    elif value_pick == 'mean':
        value_func = lambda x: x.dropna().iloc[:] if not x.dropna().empty() else None
    else:
        raise ValueError("value_pick must be 'first' or 'last' or 'mean'")

    agg_funcs = {'VALUE': value_func}
    features = chunk.groupby(['SUBJECT_ID', 'FEATURE_NAME']).agg(agg_funcs).unstack()

    # Flatten Multi-Index columns
    features.columns = [f'{i}_{j}' for i, j in features.columns]
    return features.reset_index()
    
def filter_patients_within_timeframe(data, max_hours):

    max_seconds = max_hours * 3600
    # Mark patients to keep based on the condition
    keep_patients = []
    for i, row in data.iterrows():
        if pd.notnull(row['DEATHTIME']):
            time_diff = (row['DEATHTIME'] - row['ADMITTIME']).total_seconds()
        else:
            time_diff = (row['DISCHTIME'] - row['ADMITTIME']).total_seconds()
        
        if time_diff > max_seconds:
            keep_patients.append(row['SUBJECT_ID'])
    
    return keep_patients
    
def process_data_with_timeframe_filtering(admit_times, discharge_times, death_times, filter_within_timeframe, max_hours, subject_ids):
    death_labels = []
    hospital_stay = []
    filtered_ids = []
    
    for admit, dis, death, subject_id in tqdm.tqdm(zip(admit_times, discharge_times, death_times, subject_ids), total=len(admit_times), desc='Processing mortality'):
        within_timeframe = False
        death_time = (death - admit).total_seconds() if pd.notnull(death) else None
        discharge_time = (dis - admit).total_seconds() if pd.notnull(dis) else None
        
        if death_time and death_time <= max_hours * 3600:
            within_timeframe = True
        elif discharge_time and discharge_time <= max_hours * 3600:
            within_timeframe = True
        
        if filter_within_timeframe and within_timeframe:
            continue  # Skip this patient
        
        # Add the patient's data
        hospital_stay.append(death_time if death_time else discharge_time)
        death_labels.append(1 if death_time else 0)
        filtered_ids.append(subject_id)
    
    return death_labels, hospital_stay, filtered_ids


# Model Operation

In [4]:
# Function for equalizing data
def equalize_data(data, labels):
    data["Death"] = labels
    data = data.sample(frac = 1) 
    true_data = data[data["Death"] == 1]
    false_data = data[data["Death"] == 0]
    true_length, false_length = len(true_data), len(false_data)
    
    # Equalizes labels
    if true_length > false_length:
        sample_fraction = false_length / true_length
        true_data = true_data.sample(frac = sample_fraction)
    else:
        sample_fraction = true_length / false_length
        false_data = false_data.sample(frac = sample_fraction)
        
    # Recombines data
    data = pd.concat((true_data, false_data), axis = 0)
    labels = list(data['Death'])
    features = data.drop(columns = ['Death'])
    return features, labels

# Obtains all filtered features and patients
def obtain_features(format_str = "%Y-%m-%d %H:%M:%S",
                max_hours =6, patient_threshold = 0.5, feature_threshold = 0.5,
                filter_within_timeframe = True):

    # Sets global variables to facilitate multiprocessing functions
    global id_conversion, num_hours
 
    # Obtain discharge/death/admission times for label processing
    data = pd.read_csv('dataset/ADMISSIONS.csv')
    chart_data = pd.read_csv('dataset/CHARTEVENTS.csv',)
    patient_data = pd.read_csv('dataset/PATIENTS.csv')[['SUBJECT_ID', 'DOB']]
    patient_data['DOB'] = pd.to_datetime(patient_data['DOB'])

    #micro_data = pd.read_csv('dataset/MICROBIOLOGYEVENTS.csv')
    #note_data = pd.read_csv('./dataset/NOTEEVENTS.csv')
    #prescription_data = pd.read_csv('dataset/PRESCRIPTIONS.csv')

    # Conversion dict for item IDs 
    conversion_data = pd.read_csv('dataset/D_ITEMS.csv')

    # Extract time-related discharge, death, and admit values and converts them to float
    data['ADMITTIME'] = pd.to_datetime(data['ADMITTIME'],)
    data['DISCHTIME'] = pd.to_datetime(data['DISCHTIME'])
    data['DEATHTIME'] = pd.to_datetime(data['DEATHTIME'],)
    
    # New logic to filter patients based on discharge/death time within the timeframe
    admit_times = data['ADMITTIME']
    mapping_discharge = {data['SUBJECT_ID']: data['ADMITTIME'] for _, data in data.iterrows()}
    discharge_times = data['DISCHTIME']
    death_times = data['DEATHTIME']

    admit_times = data['ADMITTIME']
    discharge_times = data['DISCHTIME']
    death_times = data['DEATHTIME']
    
    # Make sure 'ADMITTIME' and 'DOB' are in datetime format
    data['ADMITTIME'] = pd.to_datetime(data['ADMITTIME'])
    patient_data['DOB'] = pd.to_datetime(patient_data['DOB'])

    # Calculate age using relativedelta
    def calculate_age(row):
        return dateutil.relativedelta.relativedelta(row['ADMITTIME'], row['DOB']).years

    # Merge 'data' and 'patient_data' to get 'DOB' and 'ADMITTIME' in the same DataFrame for calculation
    patient_age_df = data.merge(patient_data, on='SUBJECT_ID',)
    patient_age_df['AGE'] = [None for _ in range(len(patient_age_df))]
    patient_age_df.drop_duplicates(inplace = True)
    
    # Apply the age calculation
    patient_age_df['AGE'] = patient_age_df.apply(calculate_age, axis=1)
    patient_age_df = patient_age_df[['SUBJECT_ID', 'AGE']].copy()

    # Filter based on discharge/death time within the timeframe and other logic...
    death_labels, hospital_stay, filtered_ids = process_data_with_timeframe_filtering(
        admit_times, discharge_times, death_times, filter_within_timeframe, max_hours, subject_ids = data['SUBJECT_ID'])

    # Ensure filtered_ids is a list of IDs that should be kept
    chart_data = chart_data[chart_data['SUBJECT_ID'].isin(filtered_ids)]
    
    mortality_labels = pd.DataFrame({
        'SUBJECT_ID': filtered_ids,
        'Death':death_labels
    })
    
    # Convert ITEMID to biomedical statistic labels
    id_conversion = {row['ITEMID']: row['LABEL'] for _, row in conversion_data.iterrows()}
    
    # Convert CHARTTIME to datetime
    chart_data['CHARTTIME'] = pd.to_datetime(chart_data['CHARTTIME'])

    # Convert times outside the loop
    chart_data['CHARTTIME'] = pd.to_datetime(chart_data['CHARTTIME'])

    # Properly adds admit times to df
    print('Adding times...')
    chart_admit = [mapping_discharge[subject_id] if subject_id in mapping_discharge else None for subject_id in chart_data['SUBJECT_ID']]
    chart_data['admit_time'] = chart_admit
    chart_data['admit_time'] = pd.to_datetime(chart_data['admit_time'], errors = 'coerce')
    chart_data = chart_data.dropna(subset = ['admit_time'])

    # Ensure datetime format and handle NaN values
    chart_data['CHARTTIME'] = pd.to_datetime(chart_data['CHARTTIME'], errors='coerce')
    chart_data = chart_data.dropna(subset=['CHARTTIME'])

    # Filter chart_data to include only entries within the specified time frame
    print('Filtering data...')
    chart_data = filter_chart_data_optimized(chart_data, data, max_hours=max_hours)
    
    print('Obtaining features...')
    feature_df = extract_features_from_chart_data(chart_data, id_conversion, num_processes=16)
    
    filtered_feature_df = filter_features_on_presence(feature_df, feature_threshold = feature_threshold, patient_threshold = patient_threshold)
    
    # Ensure SUBJECT_ID is a column if it's not already
    patient_data_filtered = filtered_feature_df.reset_index(inplace=False)
    
    # Merge with mortality labels
    patient_data = patient_data_filtered.merge(mortality_labels, on='SUBJECT_ID', how='left')

    # Merge with ages
    patient_age_df = patient_age_df.drop_duplicates()
    patient_data = patient_age_df.merge(patient_data, on='SUBJECT_ID',)

    # Drops columns with all NaN values and duplicate columns/rows
    patient_data = patient_data.dropna(axis=1, how='all')

    # Adds ICU Codes 
    coder = AutoEncoder()
    encoded_features = coder.train_VAE(num_epochs = 5)
    ICU_features = pd.DataFrame({"SUBJECT_ID":coder.subjects, "VALUE_ICU_Code":encoded_features})
    patient_data = patient_data.merge(ICU_features, on = 'SUBJECT_ID')
    
    labellers = {}
    string_columns_count = 0  # Counter for columns with string values
    for column in tqdm.tqdm(patient_data.columns, total = len(patient_data.columns), desc = 'Labelling data...'):

        # Check if there are any string values in the column
        if patient_data[column].apply(lambda x: isinstance(x, str)).any():

            try:
                # Increment the counter as this column contains string data
                string_columns_count += 1

                # Fill NaN values with a placeholder
                patient_data[column] = patient_data[column].fillna('missing')

                # Initialize LabelEncoder and transform values
                le = LabelEncoder()
                patient_data[column] = le.fit_transform(patient_data[column])
                labellers[column] = le

                # Identify the encoded value for 'missing' placeholder
                missing_label = le.transform(['missing'])[0]

                # Replace the 'missing' encoded value with NaN in the dataframe
                patient_data[column] = patient_data[column].replace(missing_label, None)
            except:
                patient_data[column] = patient_data[column].apply(lambda x: None if isinstance(x, str) else x)

    # Number of filtered patients
    print(f'{len(patient_data.columns)} features remaining after final filtration')
    print(f'{len(patient_data)} patients remaining after final filtration')
    
    # Removes certain features 
    features_to_remove = ['VALUE_Education Learner', 'VALUE_Education Method', 'VALUE_Education Readiness', 'VALUE_Education Response', 
                            'Value_Marital Status', 'VALUE_Religion', 'VALUE_Orientation', 'VALUE_Family Communication', 'VALUE_Education Barrier',
                            'VALUE_Code Status',]
    
    for feature in features_to_remove:
        try:
            patient_data = patient_data.drop(columns = [feature])
        except:
            pass

    labels = patient_data['Death']
    features = patient_data.drop(columns = ['Death', 'SUBJECT_ID'])
    scaler = StandardScaler()

    numpy_features = np.array([[item for item in row] for row in features.to_numpy()])
    numpy_features = scaler.fit_transform(numpy_features)
    imputer = MICE()
    
    print('Imputing...')
    impute_start = time.perf_counter()
    imputed_data = imputer.fit_transform(numpy_features)
    impute_end = time.perf_counter()
    print(f'MICE Imputation finished in {round((impute_end - impute_start)/60, 4)} minutes!')
    features = pd.DataFrame(imputed_data, columns=features.columns, index=features.index)
    
    return features, labels

def feature_analysis(features, labels, top_features = None):
    # Analyzes feature importance via a variety of methods and filters features if enabled
    if top_features:
        feature_names = analyze_features(features_scaled = features.to_numpy(), feature_names = list(features.columns), y = labels,
                                    return_top_features = True, top_n_features = top_features,)
        return feature_names
    else:
        analyze_features(features_scaled = features.to_numpy(), feature_names = list(features.columns), y = labels, return_top_features = False)
    
# Tests all models on the aggregated features and data
def test_models(features, # Feature df = HAVE IT FILTERED  
                labels, # Mortality labels
                feature_names = None, # Feature names 
                equalize = True): # Equalize labels  

    if feature_names != None:
        features = features[feature_names]
        
    print(f'Number of features: {len(features.columns)}')
    
    scaler = StandardScaler()
    def k_fold_validation_for_sklearn(clf, features, 
                                      labels, 
                                      name = 'Random Forest'):

        print(f'Starting analysis for {name}')
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)
        for i, (train_index, test_index) in tqdm.tqdm(enumerate(kf.split(features)), total = n_splits):
            
            X_train, X_test = features[train_index], features[test_index]
            y_train, y_test = labels[train_index], labels[test_index]
            
            # Scale features
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            
            # Fit and predict
            clf.fit(X_train, y_train)
            y_prob = clf.predict_proba(X_test)[:, 1]

           
            # Compute ROC curve and AUC for this fold
            fpr, tpr, _ = roc_curve(y_test, y_prob)
            roc_auc = skauc(fpr, tpr)
            aucs.append(roc_auc)
            
            # Interpolate all ROC curves at this points
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            plt.plot(mean_fpr, interp_tpr, alpha=0.4, label = f'split {i + 1} AUC = {round(roc_auc, 3)}')
        
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        
        # Calculate the mean AUC and std deviation
        mean_auc = np.mean(aucs)
        std_auc = np.std(aucs)
        print(mean_auc)
        plt.title(f'{name}, ROC (AUC = {mean_auc:.3f} $\pm$ {std_auc:.3f})')
        plt.legend(loc="lower right")
        plt.show()

        return clf

    # Equalizes labels if enabled
    if equalize == True:
        features, labels = equalize_data(features, labels) 
    print(f'Number of processed data samples: {len(labels)}')
    
    # Prepare the data
    features_array, labels = features.to_numpy(), np.array(labels)
    
    # Random Forest Classifier
    rf_clf = RandomForestClassifier(n_estimators=1000, random_state=42)
    rf_clf = k_fold_validation_for_sklearn(rf_clf, features_array, labels)
    
    # XGBoost Classifier
    xgb_clf = XGBClassifier(n_estimators=1000, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss',
                           name = 'XGBoost')
    xgb_clf = k_fold_validation_for_sklearn(xgb_clf, features_array, labels, name = 'XGBoost')
    
    # Custom-built network
    class NeuralNet(plit.LightningModule):
        def __init__(self, input_size, hidden_size=32):
            super(NeuralNet, self).__init__()
            
            self.network = nn.Sequential(
                nn.Linear(input_size, hidden_size * 8), nn.ReLU(),
                nn.BatchNorm1d(hidden_size * 8), nn.Dropout(0.3),
                
                nn.Linear(hidden_size * 8, hidden_size * 4), nn.ReLU(),
                nn.BatchNorm1d(hidden_size * 4), nn.Dropout(0.3),
                
                nn.Linear(hidden_size * 4, hidden_size * 2), nn.ReLU(),
                nn.BatchNorm1d(hidden_size * 2), nn.Dropout(0.3),
                
                nn.Linear(hidden_size * 2, hidden_size), nn.ReLU(),
                
                nn.Linear(hidden_size, hidden_size), nn.ReLU(),
                nn.BatchNorm1d(hidden_size), nn.Dropout(0.5),
            )

            self.final_linear = nn.Linear(hidden_size, 1)
            self.sigmoid = nn.Sigmoid()
            self.residual_transform = nn.Linear(input_size, hidden_size)
            
            # L1 and L2 regularization are defined but not used as layers
            
        def forward(self, x):
            out = self.network(x)  # Pass input through the network layers
            residual = self.residual_transform(x)  # Apply residual transformation to the original input
            out += residual  # Add residual connection
            out = self.final_linear(out)
            out = self.sigmoid(out)
            return out.squeeze()

        def training_step(self, batch, batch_idx):
            inputs, labels = batch
            outputs = self(inputs)
            loss = nn.BCELoss()(outputs, labels.view(-1))  # BCELoss for binary classification

            # Apply L1 and L2 regularization
            l1_reg = torch.tensor(0.)
            l2_reg = torch.tensor(0.)
            for param in self.parameters():
                l1_reg = l1_reg + torch.norm(param, 1)
                l2_reg = l2_reg + torch.norm(param, 2)

            # Regularization strengths need to be defined; for example:
            lambda1, lambda2 = 0.005, 0.005
            loss += lambda1 * l1_reg + lambda2 * l2_reg
            self.log("train_loss", loss)
            return loss

        def configure_optimizers(self):
            optimizer = torch.optim.AdamW(self.parameters(), lr=0.001)
            return optimizer
    
    def k_fold_validation_for_pytorch(model_class, features, labels, input_size):
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)
        plt.figure()
        for train_index, test_index in tqdm.tqdm(kf.split(features), total = n_splits,):
            # Prepare data
            X_train, X_test = features[train_index], features[test_index]
            y_train, y_test = labels[train_index], labels[test_index]
           
            # Convert to tensors
            X_train_tensor, y_train_tensor = torch.FloatTensor(X_train), torch.FloatTensor(y_train)
            X_test_tensor, y_test_tensor = torch.FloatTensor(X_test), torch.FloatTensor(y_test)
            
            # DataLoader setup
            train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
            train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
            
            # Model setup
            model = model_class(input_size=input_size)
            trainer = plit.Trainer(max_epochs=20)
        
            trainer.fit(model, train_loader)
            
            # Evaluate
            model.eval()
            with torch.no_grad():
                y_prob = model(X_test_tensor).numpy()
            
            # Compute ROC curve and AUC for this fold
            fpr, tpr, _ = roc_curve(y_test, y_prob)
            roc_auc = skauc(fpr, tpr)
            aucs.append(roc_auc)
            
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            plt.plot(mean_fpr, interp_tpr, alpha=0.4)
        
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        mean_auc = np.mean(aucs)
        std_auc = np.std(aucs)
        plt.title(f'Custom Neural Network Mean ROC (AUC = {mean_auc:.3f} $\pm$ {std_auc:.3f})')
        plt.legend(loc="lower right")
        plt.show()

    return rf_clf, xgb_clf, scaler
    #print("Custom Neural Network")
    #k_fold_validation_for_pytorch(NeuralNet, features_array, labels, input_size=features.shape[1])


# Main Runtime + Config

In [ ]:
# Config options for mode
hours = 24
feature_threshold = 0.25
patient_threshold = 0.1

# Obtains features
features, labels, feature_df = obtain_features(max_hours = hours, feature_threshold = feature_threshold,
                                   patient_threshold = patient_threshold,)


In [ ]:
# Performs feature analysis
feature_names = feature_analysis(features, labels, top_features = 40)

In [ ]:
# Tests model
RF, XGB, scaler = test_models(features, labels, max_hours = hours, feature_names = feature_names)

# Inference

In [ ]:
def inference_pipeline(feature_df, 
                       feature_drop,
                       true_labels, 
                       model,
                       subject_id,
                       feature_columns, 
                       feature_vector = None):
    """
    A pipeline to perform inference for a given patient by imputing missing features
    based on similar patients in the training dataset.

    :param feature_df: Path to the feature dataframe or  file.
    :param model: A scikit-learn model or the path to a saved model file.
    :param subject_id: The subject ID for which to perform the inference.
    :param feature_columns: List of features expected by the model.
    :return: The prediction for the given subject_id.
    """

    try:
        feature_df.columns
    except:
        feature_df = pd.read_csv(feature_df)

    # Obtains a random subject from df if no feature vector
    if subject_id:
        # Populates fake subject IDs for security
        if 'SUBJECT_ID' not in feature_df.columns:
            subject_ids = [i for i in range(len(feature_df))]
            feature_df['SUBJECT_ID'] = subject_ids
    
        # Sets subjectID if unknown 
        if subject_id == 'random':
            subject_id = random.sample(list(set(feature_df['SUBJECT_ID'])), 1)[0]
            
        subject_features = feature_df[feature_df['SUBJECT_ID'] == subject_id]
        label_index = int(subject_features.index.item())
    
        # Reduce to the relevant features, handling missing data
        drop_features = int(feature_drop * len(feature_columns))
        selected_feature_columns = random.sample(feature_columns, len(feature_columns) - drop_features)
        feature_vector = subject_features[selected_feature_columns].dropna(axis=1)

    missing_features = set(feature_columns) - set(feature_vector.columns)
                               
    # Use training data to find similar patients and sample missing features
    knn = NearestNeighbors(n_neighbors=5).fit(feature_df[selected_feature_columns])
    _, indices = knn.kneighbors(feature_vector)
    similar_patients_indices = indices.flatten()
    
    for feature in missing_features:
        similar_feature_values = feature_df.iloc[similar_patients_indices][feature].dropna()
        if not similar_feature_values.empty:
            try:
                distribution = gaussian_kde(similar_feature_values)
                sampled_value = distribution.resample(1)[0]
            except:
                sampled_value = np.mean(similar_feature_values.iloc[0])
            feature_vector[feature] = sampled_value  # Assuming recent_features has one row for the subject
            
    # Prepare the feature vector for prediction
    feature_vector = np.array([feature_vector[feature].iloc[0] for feature in feature_columns]).reshape(1, -1)
    feature_vector = scaler.transform(feature_vector)
    
    # Make prediction
    prediction_probability = model.predict_proba(feature_vector)
    prediction = np.argmax(prediction_probability)

    try:
        if prediction == true_labels[label_index]:
            return prediction, 1
        else:
            return prediction, 0
    except:
        return prediction_probability[1]

In [ ]:
feature_drops = [0, 0.25, 0.5, 0.8]
sample_numbers = [100, 300, 500, 1000]
feature_vec = {}
for drop in feature_drops:
    feature_vec[drop] = []
    for sample_num in tqdm.tqdm(sample_numbers, total = len(sample_numbers), desc = f'Drop {drop}'):
        correct = 0
        for i in range(sample_num):
            prediction, value = inference_pipeline(feature_df = features,
                                        true_labels = labels,
                                        model = XGB, 
                                        subject_id = 'random', 
                                        feature_columns = feature_names,  
                                        feature_drop = drop)
            correct += value
        feature_vec[drop].append(correct/sample_num)
plt.figure()
for drop, value in feature_vec.items():
    plt.plot(sample_numbers, value, label = f'{drop}')
plt.xlabel('Sample number')
plt.legend()
plt.ylabel('Probability correct')
plt.show()
    